In [972]:
import numpy as np
import pandas as pd

In [973]:
db = pd.read_csv('regr2.csv')
db

,x1,x2,t1,t2
0,1.0,0.5,10.0,5.0
1,2.5,1.3,20.5,10.2
2,3.2,2.1,30.1,15.3


In [974]:
X = db[['x1', 'x2']].values
y = db[['t1','t2']].values
print(X)
print(y)

[[1.  0.5]
 [2.5 1.3]
 [3.2 2.1]]
[[10.   5. ]
 [20.5 10.2]
 [30.1 15.3]]


In [975]:
# Forward Pass #

def relu(x):
    return np.maximum(0, x) #works componently with numpy arrays

def identity(x):
    return x

def relu_deriv(x):
    return np.where(x > 0, 1, 0) #works componently with numpy arrays (condition, if condition true then x, else y)
    
def forward(X,w1,w2,w3):
    # X: inputs
    # w1: weights input -> hidden1 adjusted for bias
    # w2: weights hidden1 -> hidden2 adjusted for bias
    # w3: weights hidden2 -> output adjusted for bias
    X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
    h1 = relu(X @ w1)
    h1c = np.concatenate([h1, np.ones((h1.shape[0], 1))], axis=1)
    h2 = relu(h1c @ w2)
    h2c = np.concatenate([h2, np.ones((h2.shape[0], 1))], axis=1)
    o1 = identity(h2c @ w3)
    return o1, h1, h2

In [976]:
# Backward Pass #

def backward(o,t,w2,w3,h2,h1):
    # o: output of forward pass
    # t: target values
    # w2: weights hidden1 -> hidden2 removed bias
    # w3: weights hidden2 -> output removed bias
    # h2: hidden2 activations
    # h1: hidden1 activations

    delta_3 = o - t
    delta_2 = relu_deriv(h2)* (delta_3 @ w3[0:3].T) # 0:3 to remove bias
    delta_1 = relu_deriv(h1)* (delta_2 @ w2[0:3].T) # 0:3 to remove bias
    return delta_1, delta_2, delta_3

In [977]:
#Stochastic Gradient Descent #

def sgd(W,dW,eta):
    new_W = []
    for W_i, dW_i in zip(W, dW):
        new_W_i = W_i - (eta * dW_i)
        new_W.append(new_W_i)
    return new_W

In [978]:
# now we do a loop, inside it is initialized weights, foward pass, backward, and sgd

def train(X,y,eta=0.01,epochs=1000):
    #initialize weights
    w1 = np.random.randn(3, 3) # 3 inputs, 3 hidden neurons, removed bias
    w2 = np.random.randn(4,3) # 3 hidden neurons, 3 hidden neurons, removed bias
    w3 = np.random.randn(4, 2) # 3 hidden neurons, 2 outputs, removed bias

    for epoch in range(epochs):
        forward_pass = forward(X,w1,w2,w3) #output, h1, h2
        backward_pass = backward(forward_pass[0],y,w2,w3,forward_pass[2],forward_pass[1]) #delta_1, delta_2, delta_3
        sgd_pass = sgd([w1,w2[0:3],w3[0:3]],[backward_pass[0],backward_pass[1],backward_pass[2]],eta) #w1_new, w2_new, w3_new
        w1 = sgd_pass[0]
        w2 = sgd_pass[1]
        w2 = np.concatenate([w2, np.ones((1, w2.shape[1]))], axis=0)
        w3 = sgd_pass[2]
        w3 = np.concatenate([w3, np.ones((1, w3.shape[1]))], axis=0)
        #if epoch % 100 == 0:
           #print(f"Epoch {epoch}: Loss = {np.mean(np.square(forward_pass[0] - y))}")
    return forward_pass[0], w1, w2, w3

print(train(X,y,0.0001,1000)) #the code error is

(array([[ 3.80650121,  1.70956684],
       [ 8.05474011,  2.78364778],
       [10.23226566,  3.3341909 ]]), array([[ 1.88135514,  1.44602432, -1.50129068],
       [ 0.9505987 ,  0.02035986, -0.55254218],
       [-0.26463195, -1.24351345,  0.05821318]]), array([[-0.61201473,  0.28702414, -0.58296299],
       [ 0.64105289,  0.74897492,  0.92343903],
       [-0.32974014,  0.40651373, -0.79689079],
       [ 1.        ,  1.        ,  1.        ]]), array([[ 2.22903258, -0.29855913],
       [ 1.60209986,  0.4054854 ],
       [ 2.46173073,  0.22633296],
       [ 1.        ,  1.        ]]))
